## Objective
Build a machine translation model (Spainish --> English), combining sub-word modelling, CNN, and LSTM.

## Setup

In [1]:
# connect to google drive
import os
import numpy as np

# mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Check for GPU free memory
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
import psutil
import humanize
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print('='*40)
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  print('='*40)
printm() 

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 120.4 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [0]:
# change root directory such that models are saved in google drive during training
root_dir = "/content/gdrive/My Drive/NLP/Char-Based-Model"
os.chdir(root_dir)
!ls

Char-Based-LSTM.ipynb		      NMT_model
code_utils			      NMT_model.optim
en_es_data			      outputs
Machine-Translation-Char-Based.ipynb  __pycache__
MT_char.ipynb			      sanity_check_en_es_data
NMT_char_model.pt		      Tensor_file.log
NMT_char_model.pt.optim


## Imports

In [0]:
# insert the path for utility custom functions
import sys
sys.path.insert(0, os.path.join(root_dir, 'code_utils'))

# custom python functions and classes
from utils import read_corpus, batch_iter, pad_sents, pad_sents_char
from vocab import Vocab, VocabEntry

In [0]:
# basic packages
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from collections import Counter, namedtuple
from docopt import docopt
from itertools import chain
import json
from typing import List, Tuple, Dict, Set, Union
from docopt import docopt

#pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence



#others
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from tqdm import tqdm
from IPython.core.debugger import set_trace

In [0]:
# Logger

import logging
logger = logging.getLogger("tensor_tracker")

file_handler = logging.FileHandler("Tensor_file.log")
stream_handler = logging.StreamHandler()
formatter = logging.Formatter('%(message)s')

file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)

logger.addHandler(file_handler)

#logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

# A helper function to check how tensor sizes change
def log_size(tsr: torch.Tensor, name: str):
    #cls = getclass()
    logger.debug(msg=f"{name} ==> size={tsr.shape}")

In [0]:
logger.debug(msg = 'a')

## Load Data and EDA

In [0]:
# load data
train_es = 'en_es_data/train.es'
train_en = 'en_es_data/train.en'

dev_es = 'en_es_data/dev.es'
dev_en = 'en_es_data/dev.en'

test_es = 'en_es_data/test.es'
test_en = 'en_es_data/test.en'


train_data_src = read_corpus(train_es, source='src')
train_data_tgt = read_corpus(train_en, source='tgt')

dev_data_src = read_corpus(dev_es, source='src')
dev_data_tgt = read_corpus(dev_en, source='tgt')

test_data_src = read_corpus(test_es, source='src')
test_data_tgt = read_corpus(test_en, source='tgt')

train_data = list(zip(train_data_src,train_data_tgt))
dev_data = list(zip(dev_data_src,dev_data_tgt))
test_data = list(zip(test_data_src,test_data_tgt))

#
print("=="*40)
print("Number of examples in train: {}".format(len(train_data)))
print("Number of examples in valid: {}".format(len(dev_data)))
print("Number of examples in test: {}".format(len(test_data)))
#
print("=="*40)
print("Spanish --> English")
es, en = next(iter(dev_data))
print("Sp: {}".format(' '.join(es)))
print("En: {}".format(' '.join(en)))
print("=="*40)


## Build Vocab
# Build Vocab with train set

size = 50000
freq_cutoff= 2
vocab_file = 'en_es_data/vocab.json'

vocab = Vocab.build(train_data_src, train_data_tgt, size, freq_cutoff)
print('generated vocabulary, source %d words, target %d words' % (len(vocab.src), len(vocab.tgt)))

vocab.save(vocab_file)
print('vocabulary saved to %s' % vocab_file)

#
print("=="*40)
print('Note that the <s> and </s> tokens are added while vocab\
      initialization.\nThese tokens are also present in target\
      top frequent words. \nThat is why vocab size for target language is lesser by 2.')
print("=="*40)


# Check tokenization process
print("=="*40)
sents = [['I', 'asgjsssd', 'will', 'be', 'there', 'for', 'you.'], ['This', 'is', 'spartaaaaaaaa.']]
print("Tokenize:\n {} \n {}\n".format(' '.join(sents[0]), ' '.join(sents[1])))

print(vocab.tgt.to_input_tensor(sents, "cpu"))
#
print("=="*40)
print("Note that 3 and 0  are <unk> and <pad> tokens!")
print("=="*40)

Number of examples in train: 216617
Number of examples in valid: 851
Number of examples in test: 8064
Spanish --> English
Sp: El ao pasado proyect estas dos diapositivas para demostrar que la capa de hielo rtico, que durante los ltimos tres millones de aos ha sido del tamao de los 48 estados, se ha reducido en un 40 por ciento.
En: <s> Last year I showed these two slides so that  demonstrate that the arctic ice cap,  which for most of the last three million years  has been the size of the lower 48 states,  has shrunk by 40 percent. </s>
initialize source vocabulary ..
number of word types: 172418, number of word types w/ frequency >= 2: 80623
initialize target vocabulary ..
number of word types: 128873, number of word types w/ frequency >= 2: 64215
generated vocabulary, source 50004 words, target 50002 words
vocabulary saved to en_es_data/vocab.json
Note that the <s> and </s> tokens are added while vocab      initialization.
These tokens are also present in target      top frequent wor

## Char Based Embedding 

In [0]:
class CNN(nn.Module):
  """CNN part of the char-based-encoder system
  @param e_word: dimension of word embedding
  @param e_char: dimension of char embedding
  @param m_size: padded/truncated word length (e.g. number of character per word)
  @param kernel_size: convolition 1d kernel sizw
  """
  
  def __init__(self, e_char, e_word, m_size, kernel_size=5):
    
    super(CNN,self).__init__()
    
    # Attributes
    self.in_channels = e_char
    self.out_channels = e_word
    self.m_size = m_size
    self.kernel_size = kernel_size
    
    # Layers
    self.conv = nn.Conv1d(self.in_channels, self.out_channels, kernel_size)
    # input shape ==> (N, C_in, L) output shape ==> (N, C_out, N_out)
    self.maxpool = nn.MaxPool1d(kernel_size = m_size-kernel_size + 1)
    
    #torch.nn.MaxPool1d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
     
  
  def forward(self,X_reshaped):
    """First applies conv1d on X_reshaped and maxpool
    @param X_reshaped: Tensor(batch_size, e_char, m_word)
    @returns X_conv_out: Tensor(batch_size,e_word)
    """
    assert X_reshaped.size(-1) == self.m_size, "Words need to be padded/truncated to size m_word"
    X_conv = self.conv(X_reshaped) # X_conv ==> (batch_size, e_word, m_word-kernel_size + 1)
    X_conv_out = self.maxpool(X_conv) #X_conv_out ==> (batch_size, e_word)
    
    return X_conv_out.squeeze(dim=-1) # remove last dimension

In [0]:
class Highway(nn.Module):
  """Implements the highway network. It provides skip connection feature. 
     Useful for building deep networks.
  """
  def __init__(self, e_word):
    
    super(Highway,self).__init__()
    
    
    #attributes
    self.e_word = e_word # dimension of word embedding
    
    
    #layers
    self.fc_proj = nn.Linear(e_word, e_word)
    self.fc_gate = nn.Linear(e_word, e_word)
    
  def forward(self, X_conv_out):
    """ Forward function for highway
    @param X_conv: Features after Maxpooling the of convolution layer outputs ==>
    Tensor of shape (b,e_word) where e_word is embedding dimension of each word and b is 
    the batch size
    
    @returns X_highway Tensor of shape (b,e_word)
    """
    #set_trace()
    X_proj = F.relu(self.fc_proj(X_conv_out)) #(b,e_word)
    X_gate = torch.sigmoid(self.fc_gate(X_conv_out)) #(b,e_word)
    #X_gate = torch.zeros(X_conv_out.size())
    X_highway = X_gate*X_proj + (1-X_gate)*X_conv_out
    
    return X_highway

In [0]:
class ModelEmbeddings(nn.Module): 
    """
    Class that converts input words to their CNN-based embeddings.
    """
    def __init__(self, embed_size, vocab):
        """
        Init the Embedding layer for one language
        @param embed_size (int): Embedding size (dimensionality) for the output 
        @param vocab (VocabEntry): VocabEntry object. See vocab.py for documentation.
        """
        super(ModelEmbeddings, self).__init__()

        
        self.embed_size = embed_size
        self.vocab = vocab
        self.src_char_len = len(vocab.char2id)
        self.e_char = 50
        self.m_size = 21 # Maximum size of characters in a word (Should be consistent with pad_sents_char)
        self.dropout_rate = 0.3
        
        
        self.char_embeddings = nn.Embedding(self.src_char_len, self.e_char, padding_idx = vocab.char2id['<pad>'])
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.cnn = CNN(self.e_char, embed_size, self.m_size)
        self.highway = Highway(embed_size)

    def forward(self, input):
        """
        Looks up character-based CNN embeddings for the words in a batch of sentences.
        @param input: Tensor of integers of shape (sentence_length, batch_size, max_word_length) where
            each integer is an index into the character vocabulary

        @param output: Tensor of shape (sentence_length, batch_size, embed_size), containing the 
            CNN-based embeddings for each word of the sentences in the batch
        """

        output = []
        sentence_length = input.size(0)
        #set_trace()
        
        for i in range(sentence_length):
          #set_trace()
          X_t = input[i] #.squeeze(dim=0) #(batch, max_word_length)
          X_enc = self.char_embeddings(X_t) #(batch, max_word_length, e_char)
          X_reshaped = torch.transpose(X_enc,1,2) #(batch,e_char,max_word_len)
          X_conv_out = self.cnn(X_reshaped) #(batch, e_word)
          X_highway = self.highway(X_conv_out) #(batch, e_word)
          X_word = self.dropout(X_highway) #(batch, e_word)
          output.append(X_word)
          
        output = torch.stack(output, dim =0) #(sentence_length, batch, e_word)
        
        return output


## Char based Decoder

In [0]:
class CharDecoder(nn.Module):
    def __init__(self, hidden_size, char_embedding_size=50, target_vocab=None):
        """ Init Character Decoder.

        @param hidden_size (int): Hidden size of the decoder LSTM
        @param char_embedding_size (int): dimensionality of character embeddings
        @param target_vocab (VocabEntry): vocabulary for the target language. See vocab.py for details.
        """
        super(CharDecoder, self).__init__()
        
        self.target_vocab = target_vocab
        self.char_embedding_size = char_embedding_size
        self.hidden_size = hidden_size
        self.padding_index = target_vocab.char2id['<pad>']
        
        self.decoderCharEmb = nn.Embedding(len(target_vocab.char2id),char_embedding_size, padding_idx = target_vocab.char2id['<pad>'])
     
        self.char_output_projection = nn.Linear(hidden_size, len(target_vocab.char2id))
        
        self.charDecoder = nn.LSTM(input_size= char_embedding_size, hidden_size = hidden_size)
    
    def forward(self, input, dec_hidden=None):
        """ Forward pass of character decoder.

        @param input: tensor of integers, shape (length, batch)
        @param dec_hidden: internal state of the LSTM before reading the input characters. A tuple of two tensors of shape (1, batch, hidden_size)

        @returns scores: called s_t in the PDF, shape (length, batch, self.vocab_size)
        @returns dec_hidden: internal state of the LSTM after reading the input characters. A tuple of two tensors of shape (1, batch, hidden_size)
        """
        dec_input = self.decoderCharEmb(input) #(length, batch, char_embedding_size)
        dec_output, dec_hidden = self.charDecoder(dec_input, dec_hidden)
        
        ## dec_output ==> (length, batch, num_dir* hidden_size)
        ## Score compute
        
        scores = self.char_output_projection(dec_output) #scores ==> (length, batch, char_vocab_size)
        
        return scores, dec_hidden


    def train_forward(self, char_sequence, dec_hidden=None):
        """ Forward computation during training.

        @param char_sequence: tensor of integers, shape (length, batch). Note that "length" here and in forward() need not be the same.
        @param dec_hidden: initial internal state of the LSTM, obtained from the output of the word-level decoder. 
         A tuple of two tensors of shape (1, batch, hidden_size)

        @returns The cross-entropy loss, computed as the *sum* of cross-entropy losses of all the words in the batch.
        """

        ### char_sequence corresponds to the sequence x_1 ... x_{n+1} from the handout (e.g., <START>,m,u,s,i,c,<END>).
        
        input_dec = char_sequence[:-1,:] #(length-1,batch)
        targets = char_sequence[1:,:] #(length-1,batch)
        loss = nn.CrossEntropyLoss(reduction='sum', ignore_index = self.padding_index)
        
        #set_trace()
        scores, last_dec = self.forward(input_dec, dec_hidden)
        cum_loss = 0 
        #scores ==> (length-1,batch, vocab_size)
        for i in range(len(scores)):
          preds = scores[i] #(batch, vocab_size)
          target = targets[i] #(batch)
          loss_t = loss(preds,target) #number
          #set_trace()
          cum_loss += loss_t
        #set_trace()
        
        return cum_loss
        
        ### END YOUR CODE

    def decode_greedy(self, initialStates, device, max_length=21):
        """ Greedy decoding
        @param initialStates: initial internal state of the LSTM, a tuple of two tensors of size (1, batch, hidden_size)
        @param device: torch.device (indicates whether the model is on CPU or GPU)
        @param max_length: maximum length of words to decode

        @returns decodedWords: a list (of length batch) of strings, each of which has length <= max_length.
                              The decoded strings should NOT contain the start-of-word and end-of-word characters.
        """


        ### Use target_vocab.char2id and target_vocab.id2char to convert between integers and characters
        ### Use torch.tensor(..., device=device) to turn a list of character indices into a tensor.
        ### We use curly brackets as start-of-word and end-of-word characters. That is, use the character '{' for <START> and '}' for <END>.
        ### Their indices are self.target_vocab.start_of_word and self.target_vocab.end_of_word, respectively.
        
        
        batch = initialStates[0].size(1)
        
        char_in  = X = torch.tensor([[self.target_vocab.char2id['{'] for i in range(batch)]], device=device) #(1,batch)
        
        dec_hidden = initialStates
        
        output_word = []
        
        for i in range(max_length):
          scores, dec_hidden = self.forward(char_in, dec_hidden) 
          #char_id = torch.arg# scores ==> (length = 1, batch, vocab_size)
          char_in = torch.argmax(scores, dim = -1) #(1,batch)
          output_word.append(char_in)
        
        words = torch.squeeze(torch.stack(output_word),dim=1) #(max_length,batch)
        
        final_words = []
        
        for i in range(batch):
          word_ids = words[:,i]
          word = ''
          for idx in word_ids:
            if idx == self.target_vocab.char2id['}']:
              break
            else:
              word += self.target_vocab.id2char[idx.item()]
          final_words.append(word)
        return final_words

## Machine Translation 

### Encoder

In [0]:
class Encoder(nn.Module):
  """Encodes the input sequence"""
  
  def __init__(self, embed_size, hidden_size, vocab_src, dropout_rate=0.2):
    super(Encoder, self).__init__()
    
    self.embed_size = embed_size
    self.hidden_size = hidden_size
    self.vocab = vocab_src # vocab in source language
    self.dropout_rate = dropout_rate
    
    self.encode_embeddings = ModelEmbeddings(embed_size, vocab_src)
    self.LSTM_encode = nn.LSTM(embed_size, hidden_size, bidirectional = True)
    self.h_projection = nn.Linear(2*hidden_size, hidden_size, bias = False)
    self.c_projection = nn.Linear(2*hidden_size, hidden_size, bias = False)
    self.dropout = nn.Dropout(dropout_rate)
    
  def forward(self, src_seq_padded, source_lengths):
    """ Input sequence is passed through bidirectional LSTM for encoding 
        @ param: src_seq_padded ==> Tensor (src_len, batch)
        @ Note that embed size is word embed dimension
        @ param: source_lengths: actual length of input sentences before padding
        @ returns: hidden states at each time step of sequence
        @ returns: initial hidden state for decoder
    """
    X = self.encode_embeddings(src_seq_padded)
    X = pack_padded_sequence(X, source_lengths)
    enc_hiddens, (last_hidden, last_cell) = self.LSTM_encode(X)
    
    enc_hiddens, _ = pad_packed_sequence(enc_hiddens, batch_first = True) #(batch, seq_len, hidden_size*2)
    
    last_hidden = torch.cat((last_hidden[0,:], last_cell[1,:]), dim = 1) #(batch, hideen_size*2)
    last_cell = torch.cat((last_cell[0,:],last_cell[1,:]), dim = 1) #(batch, hiddden_size*2)
    
    init_decoder_hidden = self.h_projection(last_hidden) #(batch,hidden_size)
    init_decoder_cell = self.c_projection(last_cell)
    
    decoder_state = (init_decoder_hidden, init_decoder_cell)
    
    return enc_hiddens, decoder_state
    

In [0]:
## Test encoder
embed_size = 5
hidden_size = 10
vocab_src = VocabEntry()
#len(vocab_src.char2id)
sent_test = [['hi', 'i', 'can', 'go'], ['do', 'it']]
sent_lens = [4,2]
sent_padded = vocab_src.to_input_tensor_char(sent_test, device = torch.device("cpu"))
#sent_padded.shape #(4,2,21)
encoder_model = Encoder(embed_size, hidden_size, vocab_src, dropout_rate=0.2)
enc_hiddens, decoder_state = encoder_model(sent_padded, sent_lens)

print('enc_hiddens shape = {}, shape expected = {}'.format(enc_hiddens.shape, (2,4,20)))
print('decoder_state[0] shape = {}, shape expected = {}'.format(decoder_state[0].shape, (2,10)))

enc_hiddens shape = torch.Size([2, 4, 20]), shape expected = (2, 4, 20)
decoder_state[0] shape = torch.Size([2, 10]), shape expected = (2, 10)


### Decoder 

In [0]:
class Decoder(nn.Module):
  """ Decodes character by character """
  def __init__(self, embed_size, hidden_size, vocab_tgt, dropout_rate=0.2):
    super(Decoder, self).__init__()
    
    # attributes
    self.embed_size = embed_size
    self.hidden_size = hidden_size
    self.vocab = vocab_tgt
    self.dropout_rate = dropout_rate
    
    # layers
    self.decode_embeddings = ModelEmbeddings(embed_size, vocab_tgt)
    self.LSTM_decoder = nn.LSTMCell(embed_size+hidden_size, hidden_size)
    self.att_projection = nn.Linear(2*hidden_size, hidden_size, bias = False)
    self.combined_output_projection = nn.Linear(3*hidden_size, hidden_size, bias= False)
    self.dropout = nn.Dropout(dropout_rate)
    
  def forward(self, enc_hiddens, enc_masks, dec_state, target_padded):
    """ Decodes character by character
        @ param: enc_hiddens ==> Tensor (batch, src_len, 2*hidden_size)
        @ param: enc_masks ==> Tensor (batch, src_len) # mask over pad tokens (no attention given to masks)
        @ param: dec_state ==> Tuple (Tensor, Tensor) ==> Initial state of decoder (hidden_state, cell_state) #(batch, hidden_size)
        @ param: target_padded ==> gold stadard targets given to compute loss: Tensor (tgt_len, batch, max_word_len)
        @ return: combined_outputs ==> Tensor (tgt_len, batch, hidden_size)
    """
    # Chop the <End> token for max length sentences
    target_padded = target_padded[:-1]
    # initialize first combined output vector o_{t-1} as zeros
    batch_size = enc_hiddens.size(0)
    #set_trace()
    o_prev = torch.zeros(batch_size, self.hidden_size, device = self.att_projection.weight.device)
    
    # initialize a list that can be used to collect combined output at each step
    combined_outputs = []
    
    enc_hiddens_proj = self.att_projection(enc_hiddens) #(batch, len_src, hidden_size)
    Y =  self.decode_embeddings(target_padded) #(batch, len_tgt, embed_size)
    
    # split Y at each time step
    Y_splited = torch.split(Y, 1, dim = 0) # return tuple with Tensor elements of shape (1, batch, embed_size)
    
    tgt_len = target_padded.size(0)
    
    # start decoding
    for i in range(tgt_len):
      Y_t = torch.squeeze(Y_splited[i], dim=0) #(batch, embed_size)
      Ybar_t = torch.cat((Y_t, o_prev), dim =1) #(batch, embed_size + hidden_size) 
      dec_state, o_prev = self.step(Ybar_t, dec_state, enc_hiddens, enc_hiddens_proj, enc_masks)
      combined_outputs.append(o_prev)
    
    combined_outputs = torch.stack(combined_outputs)
    
    return combined_outputs
      
  ## Perform global attention mechanism
  def step(self, Ybar_t, dec_state, enc_hiddens, enc_hiddens_proj, enc_masks):
    """ Returns combined output for current time step 
        @ param: Ybar_t ==> Tensor(batch, hidden_size+embed_size)
        @ param: dec_state ==> tuple(Tensor, Tensor) each Tensor of size (batch, hidden_size)
        @ param: enc_hiddens ==> Tensor (batch, len_src, 2*hidden_size)
        @ param: enc_hiddens_proj ==> Tensor (batch, len_src, hidden_size)
        @ param: enc_masks ==>  Tensor of sequence masks (batch, len_src)

        @ returns: dec_state ==> tuple(Tensor,Tensor) each Tensor (batch, hidden_size)
        @ returns: combined_output ==> Tensor (batch, hidden_size) # combined output at step t
    """

    combined_output = None

    ## compute attention scores
    dec_state = self.LSTM_decoder(Ybar_t, dec_state)
    dec_hidden, dec_cell = dec_state

    aug_dec_hidden = torch.unsqueeze(dec_hidden, dim=2) #(batch, hidden_size, 1)
    e_t = torch.bmm(enc_hiddens_proj, aug_dec_hidden) #(batch, len_src, hidden_size) * (batch, hidden_size, 1) ==> (batch, len_src, 1)
    e_t = torch.squeeze(e_t, dim =-1) #(batch, len_src)

    ## Set e_t to -inf if enc_mask has 1 (No attention to padded values)
    if enc_masks is not None:
      e_t.data.masked_fill_(enc_masks.byte(), -float('inf'))

    ## take softmax to get attention weights
    alpha_t = F.softmax(e_t, dim = 1) # (batch, len_src)
    aug_alpha_t = torch.unsqueeze(alpha_t, dim =-1) # (batch, len_src, 1)
    tr_hiddens = enc_hiddens.transpose(1,2) #(batch, 2*hidden_size, len_src)
    a_t = torch.bmm(tr_hiddens, aug_alpha_t) #(batch, 2*hidden_size,1)
    a_t = torch.squeeze(a_t, dim=-1) #(batch, 2*hidden_size)

    ## combined output
    U_t = torch.cat((a_t, dec_hidden), dim=1) #(batch, 3*hidden_size)
    V_t = self.combined_output_projection(U_t) #(batch, hidden_size)
    combined_output  = self.dropout(torch.tanh(V_t)) #(batch, hidden_size)

    return dec_state, combined_output

In [0]:
## test decoder model
embed_size = 5
hidden_size = 10

vocab_src = VocabEntry()
vocab_tgt = VocabEntry()

sent_src = [['Me', 'gustaria', 'una', 'no', 'cerveza'], ['Me', 'llamo', 'Mondly']]
sent_tgt = [['hi', 'i', 'can', 'go'], ['do', 'it']]

source_padded = vocab_src.to_input_tensor_char(sent_src, device = torch.device("cpu"))
sent_lens = [5,3]

target_padded = vocab_tgt.to_input_tensor_char(sent_tgt, device = torch.device("cpu"))

# source_padded.shape #(5,2,21)
# target_padded.shape #(4,2,21)

encoder_model = Encoder(embed_size, hidden_size, vocab_src, dropout_rate=0.2)
decoder_model = Decoder(embed_size, hidden_size, vocab_src, dropout_rate=0.2)

enc_hiddens, dec_state = encoder_model(source_padded, sent_lens)


print('enc_hiddens shape = {}, shape expected = {}'.format(enc_hiddens.shape, (2,5,20)))
print('decoder_state[0] shape = {}, shape expected = {}'.format(dec_state[0].shape, (2,10)))

## Decode 
enc_masks = torch.zeros(enc_hiddens.size(0), enc_hiddens.size(1), dtype=torch.float)
outputs = decoder_model(enc_hiddens, enc_masks, dec_state, target_padded)

##
print('output shape = {}, shape expected = {}'.format(outputs.shape, (3,2,10))) # len_tgt-1, batch, hidden_size

enc_hiddens shape = torch.Size([2, 5, 20]), shape expected = (2, 5, 20)
decoder_state[0] shape = torch.Size([2, 10]), shape expected = (2, 10)
output shape = torch.Size([3, 2, 10]), shape expected = (3, 2, 10)


### Machine Translation Model

In [0]:
class NMT_char(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab, dropout_rate = 0.2, char_decoder = True):
    super(NMT_char, self).__init__()
    
    # attributes
    self.hidden_size = hidden_size
    self.embed_size = embed_size
    self.vocab = vocab
    self.dropout_rate = dropout_rate
    
    # layers/models
    self.encoder = Encoder(embed_size, hidden_size, vocab.src, dropout_rate)
    self.decoder = Decoder(embed_size, hidden_size, vocab.tgt, dropout_rate)
    
    self.target_vocab_projection = nn.Linear(hidden_size, len(vocab.tgt), bias=False)
    self.dropout = nn.Dropout(dropout_rate)
    
    if char_decoder:
      self.charDecoder = CharDecoder(hidden_size, target_vocab = vocab.tgt)
    else:
      self.charDecoder = None
      
  def forward(self, source, target):
    """
    Machine translation model.
    @ param: source: source sentences List[List [st]]
    @ param target: target sentences List[List[str]]
    """
    source_lengths = [len(s) for s in source]
    
    target_padded = self.vocab.tgt.to_input_tensor(target, device = self.device) # Tensor: (tgt_len, batch)
    source_padded_chars = self.vocab.src.to_input_tensor_char(source, device = self.device) # Tensor: (src_len, batch, max_word_len)
    target_padded_chars = self.vocab.tgt.to_input_tensor_char(target, device = self.device) # Tensor: (tgt_len, batch, max_word_len)
    
    enc_hiddens, dec_init_state = self.encoder(source_padded_chars, source_lengths)
    enc_masks = self.generate_sent_masks(enc_hiddens, source_lengths)
    combined_outputs = self.decoder(enc_hiddens, enc_masks, dec_init_state, target_padded_chars)
    
    ##
    #set_trace()
    token_scores = self.target_vocab_projection(combined_outputs) #(tgt_len, batch, tgt_vocab_size))
    P = F.log_softmax(token_scores, dim = -1) #(tgt_len, batch, tgt_vocab_size))
    
    # Zero out, probabilities for which we have nothing in the target text
    target_masks = (target_padded != self.vocab.tgt['<pad>']).float()
    
    # Compute log probability of generating true target words
    target_gold_words_log_prob = torch.gather(P, index=target_padded[1:].unsqueeze(-1), dim=-1).squeeze(-1) * target_masks[1:]
    scores = target_gold_words_log_prob.sum() 
    
    if self.charDecoder is not None:
      max_word_len = target_padded_chars.shape[-1]
      #target_words = target_padded[1:].contiguous().view(-1,max_word_len)
      target_chars = target_padded_chars[1:].contiguous().view(-1,max_word_len) #(tgt_len-1*batch, max_word_len)
      target_outputs = combined_outputs.view(-1, self.hidden_size) # (tgt_len-1*batch, hidden_size)
      
      target_chars_oov = target_chars #torch.index_select(target_chars, dim=0, index=oovIndices)
      rnn_states_oov = target_outputs #torch.index_select(target_outputs, dim=0, index=oovIndices)
      oovs_losses = self.charDecoder.train_forward(target_chars_oov.t(), (rnn_states_oov.unsqueeze(0), rnn_states_oov.unsqueeze(0)))
      scores = scores - oovs_losses

    return scores
  
  

  def generate_sent_masks(self, enc_hiddens: torch.Tensor, source_lengths: List[int]) -> torch.Tensor:
      """ Generate sentence masks for encoder hidden states.

      @param enc_hiddens (Tensor): encodings of shape (b, src_len, 2*h), where b = batch size,
                                   src_len = max source length, h = hidden size.
      @param source_lengths (List[int]): List of actual lengths for each of the sentences in the batch.

      @returns enc_masks (Tensor): Tensor of sentence masks of shape (b, src_len),
                                  where src_len = max source length, h = hidden size.
      """
      enc_masks = torch.zeros(enc_hiddens.size(0), enc_hiddens.size(1), dtype=torch.float)
      for e_id, src_len in enumerate(source_lengths):
          enc_masks[e_id, src_len:] = 1
          
      return enc_masks.to(self.device)
    
      
  # Beam search
    
  def beam_search(self, src_sent, beam_size =5, max_decoding_time_step =70):
      """ Given a single source sentence, perform beam search, yielding translations in the target language.
      @param src_sent (List[str]): a single source sentence (words)
      @param beam_size (int): beam size
      @param max_decoding_time_step (int): maximum number of time steps to unroll the decoding RNN
      @returns hypotheses (List[Hypothesis]): a list of hypothesis, each hypothesis has two fields:
              value: List[str]: the decoded target sentence, represented as a list of words
              score: float: the log-likelihood of the target sentence
      """

      #self.device = self.att_projection.weight.device
      Hypothesis = namedtuple('Hypothesis', ['value', 'score'])
      src_sents_var = self.vocab.src.to_input_tensor_char([src_sent], self.device) #(len_src, 1, max_word_len)

      src_encodings, dec_init_vec = self.encoder(src_sents_var, [len(src_sent)]) # src_encodings => (1, len_src, hidden_size*2)
      src_encodings_att_linear = self.decoder.att_projection(src_encodings) # (1, len_src, hidden_size*2)

      h_tm1 = dec_init_vec #(tensor, tensor) => tensor (1,hidden_size)
      att_tm1 = torch.zeros(1, self.hidden_size, device=self.device)

      eos_id = self.vocab.tgt['</s>']

      hypotheses = [['<s>']]
      hyp_scores = torch.zeros(len(hypotheses), dtype=torch.float, device=self.device)
      completed_hypotheses = []


      t = 0
      while len(completed_hypotheses) < beam_size and t < max_decoding_time_step:
          t += 1
          hyp_num = len(hypotheses)
          #set_trace()

          exp_src_encodings = src_encodings.expand(hyp_num, src_encodings.size(1), src_encodings.size(2)) # share decoder state across current hypotheses

          exp_src_encodings_att_linear = src_encodings_att_linear.expand(hyp_num, src_encodings_att_linear.size(1), src_encodings_att_linear.size(2))

          y_tm1 = self.vocab.tgt.to_input_tensor_char(list([hyp[-1]] for hyp in hypotheses), device=self.device) # get last word in hypotheses (1,len(hypotheses)) --> (len(hypotheses), 1, max_word_length)
          y_t_embed = self.decoder.decode_embeddings(y_tm1) #(len(hypotheses), 1, embed_size)
          y_t_embed = torch.squeeze(y_t_embed, dim=0) #(len(hypotheses), embed_size)


          x = torch.cat([y_t_embed, att_tm1], dim=-1)

          (h_t, cell_t), att_t = self.decoder.step(x, h_tm1, exp_src_encodings, exp_src_encodings_att_linear, enc_masks=None)

          # log probabilities over target words
          log_p_t = F.log_softmax(self.target_vocab_projection(att_t), dim=-1)

          live_hyp_num = beam_size - len(completed_hypotheses)
          contiuating_hyp_scores = (hyp_scores.unsqueeze(1).expand_as(log_p_t) + log_p_t).view(-1)
          top_cand_hyp_scores, top_cand_hyp_pos = torch.topk(contiuating_hyp_scores, k=live_hyp_num)

          prev_hyp_ids = top_cand_hyp_pos / len(self.vocab.tgt)
          hyp_word_ids = top_cand_hyp_pos % len(self.vocab.tgt)

          new_hypotheses = []
          live_hyp_ids = []
          new_hyp_scores = []
          
          #set_trace()

          decoderStatesForUNKsHere = []
          for prev_hyp_id, hyp_word_id, cand_new_hyp_score in zip(prev_hyp_ids, hyp_word_ids, top_cand_hyp_scores):
              prev_hyp_id = prev_hyp_id.item()
              hyp_word_id = hyp_word_id.item()
              cand_new_hyp_score = cand_new_hyp_score.item()

              hyp_word = self.vocab.tgt.id2word[hyp_word_id]

              # Record output layer in case UNK was generated
              if hyp_word == "<unk>":
                hyp_word = "<unk>"+str(len(decoderStatesForUNKsHere))
                decoderStatesForUNKsHere.append(att_t[prev_hyp_id])

              new_hyp_sent = hypotheses[prev_hyp_id] + [hyp_word]
              if hyp_word == '</s>':
                  completed_hypotheses.append(Hypothesis(value=new_hyp_sent[1:-1],
                                                         score=cand_new_hyp_score))
              else:
                  new_hypotheses.append(new_hyp_sent)
                  live_hyp_ids.append(prev_hyp_id)
                  new_hyp_scores.append(cand_new_hyp_score)
                  
          #set_trace()

          if len(decoderStatesForUNKsHere) > 0 and self.charDecoder is not None: # decode UNKs
              decoderStatesForUNKsHere = torch.stack(decoderStatesForUNKsHere, dim=0)
              decodedWords = self.charDecoder.decode_greedy((decoderStatesForUNKsHere.unsqueeze(0), \
                                                             decoderStatesForUNKsHere.unsqueeze(0)), max_length=21, device=self.device)
              assert len(decodedWords) == decoderStatesForUNKsHere.size()[0], "Incorrect number of decoded words"
              for hyp in new_hypotheses:
                if hyp[-1].startswith("<unk>"):
                      hyp[-1] = decodedWords[int(hyp[-1][5:])]#[:-1]

          if len(completed_hypotheses) == beam_size:
              break

          live_hyp_ids = torch.tensor(live_hyp_ids, dtype=torch.long, device=self.device)
          h_tm1 = (h_t[live_hyp_ids], cell_t[live_hyp_ids])
          att_tm1 = att_t[live_hyp_ids]

          hypotheses = new_hypotheses
          hyp_scores = torch.tensor(new_hyp_scores, dtype=torch.float, device=self.device)
      
      #set_trace()
      
      if len(completed_hypotheses) == 0:
          completed_hypotheses.append(Hypothesis(value=hypotheses[0][1:],
                                                 score=hyp_scores[0].item()))

      completed_hypotheses.sort(key=lambda hyp: hyp.score, reverse=True)
      
      return completed_hypotheses
    

  @property
  def device(self):
      """ Determine which device to place the Tensors upon, CPU or GPU.
      """
      return self.target_vocab_projection.weight.device

  @staticmethod
  def load(model_path: str, char_decoder=True):
      """ Load the model from a file.
      @param model_path (str): path to model
      """
      params = torch.load(model_path, map_location=lambda storage, loc: storage)
      args = params['args']
      model = NMT(vocab=params['vocab'], char_decoder= char_decoder, **args)
      model.load_state_dict(params['state_dict'])

      return model

  def save(self, path: str):
      """ Save the odel to a file.
      @param path (str): path to the model
      """
      print('save model parameters to [%s]' % path)

      params = {
          'args': dict(embed_size=self.embed_size, hidden_size=self.hidden_size, dropout_rate=self.dropout_rate),
          'vocab': self.vocab,
          'state_dict': self.state_dict()
      }

      torch.save(params, path)    

## Training

In [0]:
## Evaluation metric
def evaluate_ppl(model, dev_data, batch_size=32):
    """ Evaluate perplexity on dev sentences
    @param model (NMT): NMT_char Model
    @param dev_data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (batch size)
    @returns ppl (perplixty on dev sentences)
    """
    was_training = model.training
    model.eval()

    cum_loss = 0.
    cum_tgt_words = 0.

    # no_grad() signals backend to throw away all gradients
    with torch.no_grad():
        for src_sents, tgt_sents in batch_iter(dev_data, batch_size):
            loss = -model(src_sents, tgt_sents).sum()

            cum_loss += loss.item()
            tgt_word_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
            cum_tgt_words += tgt_word_num_to_predict

        ppl = np.exp(cum_loss / cum_tgt_words)

    if was_training:
        model.train()

    return ppl

In [0]:
## Train

def train_model(model, optimizer, clip_grad =5.0, max_epoch =30, max_patience = 3, max_trial = 3, lr_decay = 0.5, train_batch_size = 64, log_every = 100, valid_niter = 1000):
  
  
  print('Training begins...')
  ## Temp variables
  num_trial = 0
  train_iter = patience = cum_loss = report_loss = cum_tgt_words = report_tgt_words = 0
  cum_examples = report_examples  = valid_num = 0
  hist_valid_scores = []
  train_time = begin_time = time.time()
  
  # put the model in training mode
  model.train()
  
  # iterate over the epochs
  for epoch in range(max_epoch):
    for src_sents, tgt_sents in batch_iter(train_data, batch_size=train_batch_size, shuffle=True):
        
        train_iter += 1
        optimizer.zero_grad()
        batch_size = len(src_sents)
        
        example_losses = -model(src_sents, tgt_sents)
        batch_loss = example_losses.sum()
        loss = batch_loss/batch_size
        loss.backward() # autograd
        
        # Clip gradient
        grad_norn = torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
        optimizer.step() # update parameters
        
        batch_losses_val = batch_loss.item()
        report_loss += batch_losses_val
        cum_loss += batch_losses_val
        
        tgt_words_num_to_predict = sum(len(s[1:]) for s in tgt_sents)  # omitting leading `<s>`
        report_tgt_words += tgt_words_num_to_predict
        cum_tgt_words += tgt_words_num_to_predict
        report_examples += batch_size
        cum_examples += batch_size
        
        # print interim report about training
        
        if train_iter % log_every == 0:
            #set_trace()
            print('| Epoch %d, Iter %d| Avg Loss = %.2f| Avg. ppl = %.2f| Speed %.2f words/sec| Time %.2f min|' % (epoch+1, train_iter, report_loss / report_examples, math.exp(report_loss / report_tgt_words),
                                                                                     report_tgt_words / (time.time() - train_time), (time.time() - begin_time)/60.0))

            train_time = time.time()
            report_loss = report_tgt_words = report_examples = 0.
        
        # validation
        if train_iter % valid_niter == 0:
            
            print('| <Train Summary> | Epoch %d, Iter %d| Cum. loss = %.2f| Cum. ppl = %.2f|' % (epoch+1, train_iter, cum_loss / cum_examples, np.exp(cum_loss / cum_tgt_words)))

            cum_loss = cum_examples = cum_tgt_words = 0.
            valid_num += 1

            print('Report on validation set:', file=sys.stderr)

            # compute dev. ppl and bleu
            dev_ppl = evaluate_ppl(model, dev_data, batch_size=128)   # dev batch size can be a bit larger
            valid_metric = -dev_ppl

            print('Validation:  Dev. ppl = %f' % (dev_ppl), file=sys.stderr)

            
            # learning rate scheduling
            
            is_better = (len(hist_valid_scores) == 0 or valid_metric > max(hist_valid_scores))
            hist_valid_scores.append(valid_metric)

            if is_better:
                patience = 0
                print('Save currently the best model to [%s]' % model_save_path, file=sys.stderr)
                model.save(model_save_path)

                # also save the optimizers' state
                torch.save(optimizer.state_dict(), model_save_path + '.optim')
                
            elif patience < int(max_patience):
                patience += 1
                print('Hit patience %d' % patience, file=sys.stderr)

                if patience == int(max_patience):
                    num_trial += 1
                    print('Hit #%d trial' % num_trial, file=sys.stderr)
                    
                    if num_trial == int(max_trial):
                        print('early stop!', file=sys.stderr)
                        return

                    # decay lr, and restore from previously best checkpoint
                    lr = optimizer.param_groups[0]['lr'] * float(lr_decay)
                    print('load previously best model and decay learning rate to %f' % lr, file=sys.stderr)

                    # load model
                    params = torch.load(model_save_path, map_location=lambda storage, loc: storage)
                    model.load_state_dict(params['state_dict'])
                    model = model.to(device)

                    print('restore parameters of the optimizers', file=sys.stderr)
                    optimizer.load_state_dict(torch.load(model_save_path + '.optim'))

                    # set new lr
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = lr

                    # reset patience
                    patience = 0

            if epoch +1 == int(max_epoch):
                print('Training stopped <-> Reached maximum number of epochs!', file=sys.stderr)
                return


In [0]:
# initialize the model
model = NMT_char(embed_size= 256, hidden_size=256, dropout_rate=0.3, vocab=vocab)

# define model saving path
model_save_path = 'NMT_char_model.pt'

# transfer the model to cuda if available
device = torch.device("cuda:0" if torch.cuda.device_count()>0 else "cpu")
print('Use device: %s' % device, file=sys.stderr)
model = model.to(device)

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# parameter initialization
uniform_init = 0.1
if np.abs(uniform_init) > 0.:
  print('uniformly initialize parameters [-%f, +%f]' % (uniform_init, uniform_init), file=sys.stderr)
  for p in model.parameters():
    p.data.uniform_(-uniform_init, uniform_init)


# train parameters
max_epoch =30
train_batch_size = 64

# train the model
train_model(model, optimizer, max_epoch =max_epoch, train_batch_size = train_batch_size)

Use device: cuda:0
uniformly initialize parameters [-0.100000, +0.100000]


Training begins...
| Epoch 1, Iter 100| Avg Loss = 413.37| Avg. ppl = 16136669329.75| Speed 3308.79 words/sec| Time 0.57 min|
| Epoch 1, Iter 200| Avg Loss = 337.06| Avg. ppl = 197750203.67| Speed 3253.57 words/sec| Time 1.15 min|
| Epoch 1, Iter 300| Avg Loss = 313.76| Avg. ppl = 45282543.43| Speed 3182.33 words/sec| Time 1.74 min|
| Epoch 1, Iter 400| Avg Loss = 292.17| Avg. ppl = 16321832.60| Speed 3178.64 words/sec| Time 2.33 min|
| Epoch 1, Iter 500| Avg Loss = 283.48| Avg. ppl = 8095768.21| Speed 3226.37 words/sec| Time 2.92 min|
| Epoch 1, Iter 600| Avg Loss = 267.50| Avg. ppl = 4260720.87| Speed 3150.62 words/sec| Time 3.51 min|
| Epoch 1, Iter 700| Avg Loss = 264.25| Avg. ppl = 2643083.10| Speed 3209.91 words/sec| Time 4.11 min|
| Epoch 1, Iter 800| Avg Loss = 258.06| Avg. ppl = 1958380.35| Speed 3161.89 words/sec| Time 4.71 min|
| Epoch 1, Iter 900| Avg Loss = 248.80| Avg. ppl = 1216200.06| Speed 3142.00 words/sec| Time 5.31 min|
| Epoch 1, Iter 1000| Avg Loss = 239.33| Avg. 

Report on validation set:
Validation:  Dev. ppl = 764692.806821
Save currently the best model to [NMT_char_model.pt]


save model parameters to [NMT_char_model.pt]
| Epoch 1, Iter 1100| Avg Loss = 230.13| Avg. ppl = 528805.81| Speed 2755.74 words/sec| Time 6.57 min|
| Epoch 1, Iter 1200| Avg Loss = 228.33| Avg. ppl = 362236.16| Speed 3170.24 words/sec| Time 7.17 min|
| Epoch 1, Iter 1300| Avg Loss = 217.73| Avg. ppl = 255817.32| Speed 3169.29 words/sec| Time 7.76 min|
| Epoch 1, Iter 1400| Avg Loss = 214.80| Avg. ppl = 195466.50| Speed 3169.71 words/sec| Time 8.35 min|
| Epoch 1, Iter 1500| Avg Loss = 210.47| Avg. ppl = 147539.63| Speed 3171.74 words/sec| Time 8.95 min|
| Epoch 1, Iter 1600| Avg Loss = 208.82| Avg. ppl = 119077.53| Speed 3217.85 words/sec| Time 9.54 min|
| Epoch 1, Iter 1700| Avg Loss = 205.45| Avg. ppl = 98668.34| Speed 3160.25 words/sec| Time 10.14 min|
| Epoch 1, Iter 1800| Avg Loss = 196.66| Avg. ppl = 78043.82| Speed 3135.71 words/sec| Time 10.74 min|
| Epoch 1, Iter 1900| Avg Loss = 195.58| Avg. ppl = 64861.58| Speed 3158.60 words/sec| Time 11.33 min|
| Epoch 1, Iter 2000| Avg Lo

Report on validation set:
Validation:  Dev. ppl = 59222.300336
Save currently the best model to [NMT_char_model.pt]


...

Report on validation set:
Validation:  Dev. ppl = 347.665908
Hit patience 1


| Epoch 22, Iter 71100| Avg Loss = 85.68| Avg. ppl = 123.70| Speed 3052.36 words/sec| Time 424.03 min|
| Epoch 22, Iter 71200| Avg Loss = 81.82| Avg. ppl = 109.83| Speed 3212.31 words/sec| Time 424.61 min|
| Epoch 22, Iter 71300| Avg Loss = 82.46| Avg. ppl = 109.23| Speed 3179.11 words/sec| Time 425.20 min|
| Epoch 22, Iter 71400| Avg Loss = 84.12| Avg. ppl = 115.80| Speed 3130.33 words/sec| Time 425.81 min|
| Epoch 22, Iter 71500| Avg Loss = 84.12| Avg. ppl = 118.90| Speed 3243.65 words/sec| Time 426.38 min|
| Epoch 22, Iter 71600| Avg Loss = 84.33| Avg. ppl = 116.74| Speed 3184.65 words/sec| Time 426.98 min|
| Epoch 22, Iter 71700| Avg Loss = 84.36| Avg. ppl = 116.22| Speed 3173.55 words/sec| Time 427.57 min|
| Epoch 22, Iter 71800| Avg Loss = 84.67| Avg. ppl = 117.24| Speed 3207.11 words/sec| Time 428.17 min|
| Epoch 22, Iter 71900| Avg Loss = 83.22| Avg. ppl = 116.08| Speed 3229.40 words/sec| Time 428.74 min|
| Epoch 22, Iter 72000| Avg Loss = 84.12| Avg. ppl = 117.48| Speed 3158.9

Report on validation set:
Validation:  Dev. ppl = 348.905873
Hit patience 2


| Epoch 22, Iter 72100| Avg Loss = 83.06| Avg. ppl = 114.80| Speed 3080.82 words/sec| Time 429.95 min|
| Epoch 22, Iter 72200| Avg Loss = 84.27| Avg. ppl = 112.70| Speed 3140.02 words/sec| Time 430.55 min|
| Epoch 22, Iter 72300| Avg Loss = 84.55| Avg. ppl = 115.06| Speed 3236.56 words/sec| Time 431.14 min|
| Epoch 22, Iter 72400| Avg Loss = 84.59| Avg. ppl = 118.88| Speed 3173.78 words/sec| Time 431.73 min|
| Epoch 22, Iter 72500| Avg Loss = 84.75| Avg. ppl = 119.08| Speed 3147.89 words/sec| Time 432.33 min|
| Epoch 22, Iter 72600| Avg Loss = 84.95| Avg. ppl = 123.76| Speed 3161.49 words/sec| Time 432.93 min|
| Epoch 22, Iter 72700| Avg Loss = 84.94| Avg. ppl = 119.76| Speed 3143.33 words/sec| Time 433.53 min|
| Epoch 22, Iter 72800| Avg Loss = 84.21| Avg. ppl = 116.74| Speed 3195.43 words/sec| Time 434.12 min|
| Epoch 22, Iter 72900| Avg Loss = 83.55| Avg. ppl = 119.13| Speed 3156.83 words/sec| Time 434.71 min|
| Epoch 22, Iter 73000| Avg Loss = 86.36| Avg. ppl = 124.37| Speed 3259.6

Report on validation set:
Validation:  Dev. ppl = 355.497405
Hit patience 3
Hit #3 trial
early stop!


In [0]:
model.device

device(type='cuda', index=0)

## Evaluation

In [0]:
# Load the model
model = NMT_char(embed_size= 256, hidden_size=256, dropout_rate=0.3, vocab=vocab) # blueprint

params = torch.load(model_save_path, map_location=lambda storage, loc: storage)
model.load_state_dict(params['state_dict'])

device = torch.device("cuda:0" if torch.cuda.device_count()>0 else "cpu")
model = model.to(device)

# Evaluation mode
model.eval()

NMT_char(
  (encoder): Encoder(
    (encode_embeddings): ModelEmbeddings(
      (char_embeddings): Embedding(96, 50, padding_idx=0)
      (dropout): Dropout(p=0.3)
      (cnn): CNN(
        (conv): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
        (maxpool): MaxPool1d(kernel_size=17, stride=17, padding=0, dilation=1, ceil_mode=False)
      )
      (highway): Highway(
        (fc_proj): Linear(in_features=256, out_features=256, bias=True)
        (fc_gate): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (LSTM_encode): LSTM(256, 256, bidirectional=True)
    (h_projection): Linear(in_features=512, out_features=256, bias=False)
    (c_projection): Linear(in_features=512, out_features=256, bias=False)
    (dropout): Dropout(p=0.3)
  )
  (decoder): Decoder(
    (decode_embeddings): ModelEmbeddings(
      (char_embeddings): Embedding(96, 50, padding_idx=0)
      (dropout): Dropout(p=0.3)
      (cnn): CNN(
        (conv): Conv1d(50, 256, kernel_size=(5,), stride=(1

In [0]:
##
# beam search check
src_sent =['Tambien', 'tenemos', 'que', 'tener', 'cuidado', 'con', 'el', 'hielo', 'se', 'resbala', 'facilmente', 'en', 'el.']
en_ref = ['We', 'also', 'have', 'to', 'be', 'careful', 'with', 'the', 'ice,','it', 'slides', 'easily', 'on', 'it.']
en_hat = model.beam_search(src_sent,5,70)
print("=="*40)
print("Model Translation:\n")
print('{}'.format(' '.join(en_hat[0].value)))
print("\n")
print("Human Reference:\n")
print('{}'.format(' '.join(en_ref)))
print("=="*40)

Model Translation:

We also have to be careful with the ice in the ice faster in the ice.


Human Reference:

We also have to be careful with the ice, it slides easily on it.


In [0]:
#Now, we can have to be able to be able to see the way to be in the way in the world.

## Bleu

In [0]:
###########################
## Compute Blue using nltk 
###########################

Hypothesis = namedtuple('Hypothesis', ['value', 'score'])

def compute_corpus_level_bleu_score(references: List[List[str]], hypotheses: List[Hypothesis]) -> float:
    """ Given decoding results and reference sentences, compute corpus-level BLEU score.
    @param references (List[List[str]]): a list of gold-standard reference target sentences
    @param hypotheses (List[Hypothesis]): a list of hypotheses, one for each reference
    @returns bleu_score: corpus-level BLEU score
    """
    if references[0][0] == '<s>':
        references = [ref[1:-1] for ref in references]
    bleu_score = corpus_bleu([[ref] for ref in references],
                             [hyp.value for hyp in hypotheses])
    return bleu_score
  
###########################
## Model Beam Search for test set 
###########################



def beam_search(model: NMT_char, test_data_src: List[List[str]], beam_size: int, max_decoding_time_step: int) -> List[List[Hypothesis]]:
    """ Run beam search to construct hypotheses for a list of src-language sentences.
    @param model (NMT): NMT Model
    @param test_data_src (List[List[str]]): List of sentences (words) in source language, from test set.
    @param beam_size (int): beam_size (# of hypotheses to hold for a translation at every step)
    @param max_decoding_time_step (int): maximum sentence length that Beam search can produce
    @returns hypotheses (List[List[Hypothesis]]): List of Hypothesis translations for every source sentence.
    """
    was_training = model.training
    model.eval()

    hypotheses = []
    with torch.no_grad():
        for src_sent in tqdm(test_data_src):
            example_hyps = model.beam_search(src_sent, beam_size=beam_size, max_decoding_time_step=max_decoding_time_step)

            hypotheses.append(example_hyps)

    if was_training: model.train(was_training)

    return hypotheses


###########################
## Decoding using beam search 
###########################
  
def decode():
    """ Performs decoding on a test set, and save the best-scoring decoding results.
    If the target gold-standard sentences are given, the function also computes
    corpus-level BLEU score.
    @param args (Dict): args from cmd line
    """

    print("load test source sentences", file=sys.stderr)
    test_data_src = read_corpus(test_es, source='src')
    
    
    print("load test target sentences", file=sys.stderr)
    test_data_tgt = read_corpus(test_en, source='tgt')

    hypotheses = beam_search(model, test_data_src,
                             beam_size=5,
                             max_decoding_time_step=70)

    top_hypotheses = [hyps[0] for hyps in hypotheses]
    bleu_score = compute_corpus_level_bleu_score(test_data_tgt, top_hypotheses)
    print('Corpus BLEU: {}'.format(bleu_score * 100), file=sys.stderr)
    
    with open('model_translation.txt', 'w') as f:
        for src_sent, hyps in zip(test_data_src, hypotheses):
            top_hyp = hyps[0]
            hyp_sent = ' '.join(top_hyp.value)
            f.write(hyp_sent + '\n')

In [0]:
decode()

load test source sentences
load test target sentences
100%|██████████| 8064/8064 [09:11<00:00, 14.63it/s]
Corpus BLEU: 24.555234003232144


## Inference

In [0]:
# Read in machine translations without char decoder
with open('test_outputs_a4.txt','r') as f:
  nmt = f.readlines()
  
# NLTK
from nltk.translate import bleu  
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

In [0]:
idx = 399

print(' '.join(test_data_src[idx][:]))
print('\nRef:')
print(' '.join(test_data_tgt[idx][1:-1]))

score = corpus_bleu([test_data_src[idx]], [model.beam_search(test_data_src[idx])[0].value], smoothing_function=smoothie)
print('\nNMT_char BLEU = {:.2f}'.format(score*100))
print(' '.join(model.beam_search(test_data_src[idx][1:-1])[0].value))


score = corpus_bleu([test_data_src[idx][1:-1]], [nmt[idx][:]], smoothing_function=smoothie)
print('\nNMT BLEU = {:.2f}'.format(score*100))
print(''.join(nmt[idx][:]))



print()

Y nunca las he visto.

Ref:
And I've never seen them.

NMT_char BLEU = 0.00
I never have them.

NMT BLEU = 14.80
And I've never seen them.




In [0]:
model.beam_search(test_data_src[idx])[0].value

['How',
 'do',
 'you',
 'call',
 'the',
 'leader',
 'of',
 'a',
 'Republican',
 'country?']

## Conclusion